In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop
from utils.dataset import DataSet, LABELS
from models.textrnn import CuDNNGRULast as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn')
MODEL_PRE = 'textrnn_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')
RESULT_CSV = os.path.join(RESULTS_PATH, 'textrnn.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop(LR))

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 101s 963us/step - loss: 0.2977 - acc: 0.9097 - val_loss: 0.2051 - val_acc: 0.9420

Epoch 00001: _f1_monitor improved from -inf to 0.56967, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/textrnn_0.h5
Epoch 2/100
    64/105000 [..............................] - ETA: 1:36 - loss: 0.0705 - acc: 0.9844

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


105000/105000 [==============================] - 100s 956us/step - loss: 0.1976 - acc: 0.9417 - val_loss: 0.1888 - val_acc: 0.9449

Epoch 00002: _f1_monitor improved from 0.56967 to 0.57331, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/textrnn_0.h5
Epoch 3/100
105000/105000 [==============================] - 101s 962us/step - loss: 0.1829 - acc: 0.9449 - val_loss: 0.2054 - val_acc: 0.9419

Epoch 00003: _f1_monitor improved from 0.57331 to 0.60267, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/textrnn_0.h5
Epoch 4/100
105000/105000 [==============================] - 101s 960us/step - loss: 0.1708 - acc: 0.9477 - val_loss: 0.1921 - val_acc: 0.9454

Epoch 00004: _f1_monitor improved from 0.60267 to 0.60593, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/textrnn_0.h5
Epoch 5/100
105000/105000 [==============================] - 101s 961us/step - loss: 0.1573 - acc

105000/105000 [==============================] - 101s 961us/step - loss: 0.2275 - acc: 0.9264 - val_loss: 0.2894 - val_acc: 0.9083

Epoch 00005: _f1_monitor did not improve from 0.60616
Epoch 6/100
105000/105000 [==============================] - 101s 963us/step - loss: 0.2122 - acc: 0.9324 - val_loss: 0.2939 - val_acc: 0.9069

Epoch 00006: _f1_monitor did not improve from 0.60616

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 7/100
105000/105000 [==============================] - 101s 962us/step - loss: 0.1638 - acc: 0.9490 - val_loss: 0.3126 - val_acc: 0.9047

Epoch 00007: _f1_monitor did not improve from 0.60616
Epoch 8/100
105000/105000 [==============================] - 101s 962us/step - loss: 0.1484 - acc: 0.9542 - val_loss: 0.3261 - val_acc: 0.9027

Epoch 00008: _f1_monitor did not improve from 0.60616
Epoch 9/100
105000/105000 [==============================] - 101s 960us/step - loss: 0.1356 - acc: 0.9585 - val_loss: 0.3502 - val_acc: 0.

105000/105000 [==============================] - 101s 962us/step - loss: 0.1377 - acc: 0.9572 - val_loss: 0.2820 - val_acc: 0.9236

Epoch 00007: _f1_monitor did not improve from 0.69450

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 8/100
105000/105000 [==============================] - 101s 962us/step - loss: 0.0904 - acc: 0.9732 - val_loss: 0.3050 - val_acc: 0.9211

Epoch 00008: _f1_monitor did not improve from 0.69450
Epoch 9/100
105000/105000 [==============================] - 101s 962us/step - loss: 0.0756 - acc: 0.9781 - val_loss: 0.3261 - val_acc: 0.9203

Epoch 00009: _f1_monitor did not improve from 0.69450
Epoch 10/100
105000/105000 [==============================] - 101s 963us/step - loss: 0.0648 - acc: 0.9815 - val_loss: 0.3442 - val_acc: 0.9171

Epoch 00010: _f1_monitor did not improve from 0.69450

Epoch 00010: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 6th grain: f1_score - 0.6945001264650104, acc - 0.9

The 9th grain: f1_score - 0.6336883780751971, acc - 0.8222
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 102s 968us/step - loss: 0.5647 - acc: 0.7820 - val_loss: 0.4667 - val_acc: 0.8292

Epoch 00001: _f1_monitor improved from -inf to 0.60967, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/textrnn_10.h5
Epoch 2/100
105000/105000 [==============================] - 101s 963us/step - loss: 0.4259 - acc: 0.8436 - val_loss: 0.4269 - val_acc: 0.8433

Epoch 00002: _f1_monitor improved from 0.60967 to 0.66694, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/textrnn_10.h5
Epoch 3/100
105000/105000 [==============================] - 101s 963us/step - loss: 0.3951 - acc: 0.8559 - val_loss: 0.4195 - val_acc: 0.8435

Epoch 00003: _f1_monitor did not improve from 0.66694
Epoch 4/100
105000/105000 [==============================] - 101s 964us/step - loss

105000/105000 [==============================] - 101s 964us/step - loss: 0.2839 - acc: 0.9090 - val_loss: 0.3320 - val_acc: 0.8983

Epoch 00004: _f1_monitor did not improve from 0.70916
Epoch 5/100
105000/105000 [==============================] - 101s 963us/step - loss: 0.2617 - acc: 0.9158 - val_loss: 0.3460 - val_acc: 0.8920

Epoch 00005: _f1_monitor did not improve from 0.70916

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 6/100
105000/105000 [==============================] - 101s 965us/step - loss: 0.2097 - acc: 0.9320 - val_loss: 0.3656 - val_acc: 0.8861

Epoch 00006: _f1_monitor did not improve from 0.70916
Epoch 7/100
105000/105000 [==============================] - 101s 964us/step - loss: 0.1925 - acc: 0.9380 - val_loss: 0.3731 - val_acc: 0.8868

Epoch 00007: _f1_monitor did not improve from 0.70916
Epoch 8/100
105000/105000 [==============================] - 101s 965us/step - loss: 0.1776 - acc: 0.9431 - val_loss: 0.3890 - val_acc: 0.

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6226434047296789
The 1th f1: 0.5125698958399216
The 2th f1: 0.6784787381708217
The 3th f1: 0.6061596813913581
The 4th f1: 0.7571463147547476
The 5th f1: 0.7111596460375074
The 6th f1: 0.6945001264650104
The 7th f1: 0.7327076552717514
The 8th f1: 0.6551972491267138
The 9th f1: 0.6336883780751971
The 10th f1: 0.6669430054019951
The 11th f1: 0.6906281842332302
The 12th f1: 0.6859236723780278
The 13th f1: 0.6654285495367138
The 14th f1: 0.6409363931819809
The 15th f1: 0.6780945296250716
The 16th f1: 0.5187750159822786
The 17th f1: 0.7091600052544558
The 18th f1: 0.554051289429081
The 19th f1: 0.6699850465425525
The average f1 of val is 0.6542088390714047


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)